# Neurons & Behavior

In this notebook, we'll look at behavior in the Allen Brain Observatory two-photon imaging dataset. Specifically, we'll see if we can use the activity of neurons to predict the animal's running speed.

In [ ]:
# Typical toolboxes
import numpy as np
import pandas as pd
import scipy as sp

# Plotting setup
%config InlineBackend.figure_format = 'retina' # Improve the resolution of our plots!
%matplotlib inline
import matplotlib.pyplot as plt

## Step I. Get raw fluorescence & running traces

This dataset uses **2-photon calcium imaging** to measure neural activity in mice while they're running on a wheel and viewing visual stimuli. We don't have access to the animal's performance on the task, but we can still use the animal's movement on the wheel (a very rich measure of arousal, actually!) as an interesting behavioral output.

First, let's get some data.

In [ ]:
# Allen specific toolboxes
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

boc = BrainObservatoryCache(manifest_file='/datasets/allen-brain-observatory/visual-coding-2p/manifest.json')

# Get the drifting gratings experiments for a specific cell ID
cell_id = 662074297 #541513979
exps = boc.get_ophys_experiments(cell_specimen_ids=[cell_id], stimuli=['drifting_gratings'])

# Let's just look at the first session
session_id = exps[0]['id']
data_set = boc.get_ophys_experiment_data(session_id)

Rather than look at single cells, let's look at all of the cells in our field of view. We'll use the `get_dff_traces()` method to get the raw data traces.

<div class="alert alert-success">

<b>Task</b>: After getting the dff trace, calculate a mean, and plot the mean fluorescence versus time.

</div>

In [ ]:
# Get all of the fluorescence traces (ts = timestamps; dff = deltaF/F)
ts, dff = data_set.get_dff_traces()

# Get the average change in fluorescence

# Plot the sum of all of the fluorescence activity


That's the imaged activity of all of the cells in our field of view, but does this activity correlate with the behavior of the animal? Specifically, let's look at the mouse's running speed.

In [ ]:
# Get the running speed from this dataset (dxcm = distance in cm; ts = timestamps)
dxcm, ts = data_set.get_running_speed()

# Plot it
plt.figure(figsize=(14,5))
plt.plot(ts,dxcm)
plt.xlabel('Time (seconds)')
plt.ylabel('Speed')
plt.show()

## Step 2. Clean up the running data and visualize.

Running activity is fairly coarse, so rather than try to predict the activity at 30Hz, let's bin the activity (summed across all neurons) into bins of 5 seconds.

In [ ]:
bin_size = 5 
bin_stamps = np.arange(0,ts.max(),bin_size)
num_bins = len(bin_stamps)

run_bin = np.empty(num_bins)
response_bin = np.empty(num_bins)

for i in range(num_bins):
    
    # Get the values within our time bin and take a mean
    run_bin[i] = dxcm[np.where((ts>i*bin_size)&(ts<((i+1)*bin_size)))].mean() 
    response_bin[i] = dff_mean[np.where((ts>i*bin_size)&(ts<((i+1)*bin_size)))].mean()

If we look at the `run_bin` we just created, we'll see there are some NaN values, since the running data is recorded for slightly less time than the 2P imaging. This might cause a problem later, so let's remove them. 

<div class="alert alert-success">

<b>Task</b>: Remove the NaNs from the running trace, and remove those same data points from the summed activity trace.

</div>

In [ ]:
response_bin = response_bin[~np.isnan(run_bin)]
run_bin = run_bin[~np.isnan(run_bin)]

Plot our binned running and response traces together to see if there is, visually, any correlation.

In [ ]:
# Scale the response trace
response_bin_scaled = response_bin*1000

# Plot it
plt.figure(figsize=(14,5))
plt.plot(run_bin)
plt.plot(response_bin_scaled)
plt.xlabel('Time (seconds)')
plt.show()

## Step 3: Predict fluorescence data from mouse's running speed

It's tough to tell visually, but maybe if we run a linear regression we'll be able to see if there is a relationship. 

When we run this linear regression (using least squares method) on our data, we need to decide which data to *train* the model on, and which data to *test* it on. Let's train it on samples 200 onward, since that contains a lot of variability. We'll then see if we can use the model to predict the first 200 samples.

**Side note**: We saw `linregress()` in the correlations notebook, which can work for these purposes. If you ever neet to run a regression with multivariate data, though, you'll need [lstsq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html) in `scipy.linalg()`. See the commented lines of code below to see how this would work here. You'll notice they return similar, but slightly different, results.

In [ ]:
# Subtract the mean from both response and running
response_bin -= response_bin.mean()
run_bin -= run_bin.mean()

# Run regression on training data & determine the coefficient 
coef, intercept, r_value, p_value, std_err = sp.stats.linregress(run_bin[200:], response_bin[200:])


# Alternative way to do this below, using a slightly different function
# from scipy.linalg import lstsq
# This ',None' business is because we have to append a column of to calculate intercept
# solution, residues, rank, singular_values = lstsq(run_bin[200:,None], response_bin[200:,None])
# coef = solution[0]

print(coef)

In [ ]:
# Use the coefficient we generated to see if we can predict running activity
predict = coef*run_bin[:200]

# Plot the prediction
plt.figure(figsize=(14,5))
plt.plot(response_bin[:200])
plt.plot(predict)
plt.show()

To test how good this prediction is, we can use Pearson's *r*.

In [ ]:
pearsons_r , pearsons_p = sp.stats.pearsonr(response_bin[:200], predict)
print("pearson's r: %f    pearson's p: %f" % (pearsons_r, pearsons_p))

## Step 4. Try this on other data!

Hmm, there isn't a clear linear relationship between running and the responses of all of the cells in this experiment. However, there definitely *are* cells that have a clear relationship!

Consider the following passage from [Yavorska & Weher (2016)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5040712/):

>  In visual cortex, running depolarizes both PNs and inhibitory cells. The resulting increase in both excitation and inhibition in PNs reduces membrane potential variance, and leads to more stimulus-evoked spikes without any increase in spontaneous activity (Polack et al., 2013). Whereas cholinergic input affects membrane potential fluctuations during quiescent periods, the effect of locomotion on membrane potential variance is mostly dependent on noradrenergic input. Interestingly, SOM neurons do not show decreased membrane potential variability during running, suggesting a differential influence of norepinephrine on SOM neurons and PNs (Polack et al., 2013). Different classes of inhibitory neurons show marked differences in how they are modulated by locomotion in the visual cortex. VIP neurons are depolarized throughout the entire running period, while PV cells only respond transiently at the beginning. SOM neurons are typically suppressed during running, and fire mostly at the end of the running period (Fu et al., 2014). These results suggest that the effect of locomotion is mediated by a disinhibitory circuit, in which VIP cells inhibit SOM cells and thereby increase the activity of neighboring PNs. 

<div class="alert alert-success">
    <b>Task</b>: Check to see if there are either <b>single cells</b> in the same experiment above or <b>an experiment in a different cell type</b> have strong correlations with running by editing the code above. Remember to choose a reasonable set of the data to train your model on.</div>

If you're curious, you could also try this on the Neuropixels dataset. It will work fairly similarly! The code below will get you started, see [here](https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html) for more documentation. Once you extract a session, you can use `session.running_speed()` or `session.get_pupil_data()`. See the [Neuropixels Cheat Sheet](https://brainmapportal-live-4cc80a57cd6e400d854-f7fdcae.divio-media.net/filer_public/0f/5d/0f5d22c9-f8f6-428c-9f7a-2983631e72b4/neuropixels_cheat_sheet_nov_2019.pdf).

In [ ]:
# Import the Neuropixels Cache
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

# We have all of this data on the datahub! This is where it lives.
manifest_path = '/datasets/allen-brain-observatory/visual-coding-neuropixels/ecephys-cache/manifest.json' 

# Create the EcephysProjectCache object
cache = EcephysProjectCache.fixed(manifest=manifest_path)

# Get the sessions available in this dataset
sessions = cache.get_session_table()
print('Total number of sessions: ' + str(len(sessions)))
sessions.head()